# Intermediate Geoprocessing with Python

## Get our environment set up
 * Install Anaconda
 * Review of packmage managers (pip, conda, setuptools)
 * Using ```conda``` to install packages
 * ```arcpy``` and the magic .pth
 * Virtual Environments

## Logging
Use configuration files
 * Python: What you are already writing in, but difficult to read
 * YAML: Easiest to read, but requires extra libraries (and there is [no concensus on what to use](https://pypi.python.org/pypi?%3Aaction=search&term=yaml&submit=search))  
 * JSON: Easy to read and uses built-in libraries

In [ ]:
%%javascript
{
    "version": 1,
    "disable_existing_loggers": false,
    "formatters": {
        "simple": {
            "format": "%(asctime)s - %(name)s - %(levelname)s - %(message)s"
        },
        "statistic": {
            "format": "%(asctime)s : %(message)s"
        },
        "special": {
            "format": "%(message)s"
        }
    },
    "filters": {
          "statsfilter": {
            "()": "ext://logging_config.KeywordFilter",
            "param": "_stats_"
          }

    },

    "handlers": {
        "console": {
            "class": "logging.StreamHandler",
            "level": "DEBUG",
            "formatter": "simple",
            "stream": "ext://sys.stdout"
        },

        "info_file_handler": {
            "class": "logging.handlers.TimedRotatingFileHandler",
            "level": "INFO",
            "formatter": "simple",
            "filename": "info.log",
            "when": "D",
            "backupCount": 20,
            "encoding": "utf8"
        },

        "error_file_handler": {
            "class": "logging.handlers.TimedRotatingFileHandler",
            "level": "ERROR",
            "formatter": "simple",
            "filename": "error.log",
            "when": "D",
            "backupCount": 20,
            "encoding": "utf8"
        },

        "stats_file_handler": {
              "class": "logging.handlers.RotatingFileHandler",
              "level": "INFO",
              "formatter": "statistic",
              "filename": "stats.log",
              "maxBytes": 10485760,
              "backupCount": 20,
              "encoding": "utf8",
              "filters": ["statsfilter"]
          },

        "special_file_handler": {
              "class": "logging.handlers.RotatingFileHandler",
              "level": "DEBUG",
              "formatter": "special",
              "filename": "specialcase.log",
              "maxBytes": 10485760,
              "backupCount": 20,
              "encoding": "utf8"
          }
    },

    "loggers": {
        "statistics": {
            "level": "INFO",
            "handlers": ["stats_file_handler"],
            "propagate": 0
        },
        "special": {
            "level": "DEBUG",
            "handlers": ["special_file_handler"],
            "propagate": 0
        }
    },

    "root": {
        "level": "INFO",
        "handlers": ["console", "info_file_handler", "error_file_handler"]
    }
}

## Logging Config Anatomy
 * Level is import (DEBUG, INFO, WARNING, ERROR, CRITICAL)
 * root is the root logger
 * loggers have handlers and do the work
 * handlers have formats and filters and define the work
 
 Log records propagate upwards. Names work like namespaces.
 A logger named "myprogram.function_a" propages to a logger named "myprogram" which propagates to root.
 To stop upward propagation (don't send the log to root) set propagate to 0.  
 (Check out http://www.shutupandship.com/2012/02/how-python-logging-module-works.html)  
 

In [ ]:
# Levels are numbers
import logging
logging.DEBUG

In [ ]:
logging.INFO

In [ ]:
logging.WARNING

In [ ]:
logging.ERROR

In [ ]:
logging.CRITICAL

In [ ]:
## Reading the config (and a custom filter)

In [ ]:
import logging.config
from string import punctuation
from string import whitespace
import json
import os


class KeywordFilter(logging.Filter):
    def __init__(self, param=None):
        super(KeywordFilter, self).__init__()
        self.param = param

    def filter(self, record):
        if self.param is None:
            allow = True
        else:
            allow = record.msg.startswith(self.param)
        if allow:
            record.msg = record.msg[len(self.param):].lstrip(punctuation + whitespace)
        return allow


def setup_logging(
        default_path='logging.json',
        default_level=logging.INFO,
        env_key='LOG_CFG'
):
    """Setup logging configuration

    """
    path = default_path
    value = os.getenv(env_key, None)
    if value:
        path = value
    if os.path.exists(path):
        with open(path, 'rt') as f:
            config = json.load(f)
        logging.config.dictConfig(config)
    else:
        logging.basicConfig(level=default_level)

## Field Calculator
Download the [sample data](https://www.dropbox.com/s/ewanlg0vhm9rkv7/Assignment3.zip)
  
### Convert our zip codes with ```.format```
Need a reference? Google it! (The docs are hard to read)  

In [ ]:
# Cases: zip = 123459999, zip = '123459999', zip = '12345', zip = 12345
# Want: '12345-9999 or 12345'

Now carry over to ArcGIS Desktop, and try these cases:
 * Convert School Districts to title case
 * Convert phone numbers to ###.###.#### but leave emails alone


In [ ]:
district = "BRENTWOOD"

In [ ]:
contact2 = '314-555-1212'
contact3 = 'webmaster@gmail.com'

## Hour Two
Download the [zip code data from dropbox](https://www.dropbox.com/s/yn5usq1zenvszem/MO_1992_Zip_Code_Areas_shp.zip)  
Install requests http://www.lfd.uci.edu/~gohlke/pythonlibs/#requests  
* Requests and Web Communication: Wifi Sites Demo
* Regexp
* Basic Cursors
* Rewriting ameren.py

In [ ]:
import requests
url = "https://raw.githubusercontent.com/marigolds6/pythonintermediate/master/wifisample.js"
r = requests.get(url)

In [ ]:
r.raise_for_status()

In [ ]:
import json
nodes = r.json()

What we want:
 * Building without all the whitespace
 * Type changed to wifitype
 * build map locator

In [ ]:
for node in nodes:
     print "A type {wifitype} wifi node is located in {building} ({locator}) at {lat}, {long}".format(**node)

## Reg Exp for pattern matching
Goals:
 * Examine the Ameren webpage at http://apps.ameren.com/outage/OutagebyZip.aspx?state=MO
 * Write regexp to find the pattern in the page

In [ ]:
import re
pattern = ''
matcher = re.compile(pattern)
output = matcher.match('mystring')


Get help: https://regex101.com/ (and others)

In [ ]:
import requests
url = 'http://apps.ameren.com/outage/OutagebyZip.aspx?state=MO'
r = requests.get(url)

In [ ]:
r.content